<a href="https://colab.research.google.com/github/laliali20/Capstone/blob/main/DSICapstone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction